In [1]:
import numpy as np

In [2]:
import onnx

In [3]:
import os

In [4]:
import glob

In [5]:
from onnx import numpy_helper

In [6]:
import tvm

In [7]:
from tvm import te

In [8]:
import tvm.relay as relay

In [9]:
mnist_model = onnx.load('../mnist/mnist-8.onnx')

In [10]:
# load module param and mod

In [11]:
input_name = "Input3"

In [12]:
shape_dict = {input_name: (1, 1, 28, 28)}
print(shape_dict)

{'Input3': (1, 1, 28, 28)}


In [13]:
mod, params = relay.frontend.from_onnx(mnist_model, shape_dict)

In [33]:
# func = mod["main"]

In [34]:
# func = relay.Function(func.params, relay.nn.softmax(func.body), None, func.type_params, func.attrs)

In [14]:
# target = tvm.target.arm_cpu("rasp3b")
# target = tvm.target.create('llvm')
 target = tvm.target.Target("llvm -device=arm_cpu -mtriple=aarch64-linux-gnu -mattr=+neon")

In [15]:
print(target)

llvm -keys=arm_cpu,cpu -device=arm_cpu -link-params=0 -mattr=+neon -mtriple=aarch64-linux-gnu


In [20]:
with tvm.transform.PassContext(opt_level=2):
# with relay.build_config(opt_level=2):    
    mod_lib = relay.build(mod, target, params=params, mod_name='mnist')

In [21]:
print(mod_lib)

In [22]:
# lib export
lib_path = "../lib/mnist.so"

In [24]:
mod_lib.export_library(lib_path, cc="/usr/bin/aarch64-linux-gnu-g++")
# cc="/usr/bin/arm-linux-gnueabihf-g++

In [ ]:
# params export

In [25]:
param_path = "../lib/mnist.params"

In [26]:
with open(param_path, 'wb') as fo:
    fo.write(relay.save_param_dict(params))

In [27]:
# test module

In [28]:
test_data_dir = '../mnist/test_data_set_2'

In [29]:
inputs = []

In [30]:
inputs_num = len(glob.glob(os.path.join(test_data_dir, 'input_*.pb')))

In [31]:
for i in range(inputs_num):
    input_file = os.path.join(test_data_dir, 'input_{}.pb'.format(i))
    tensor = onnx.TensorProto()
    with open(input_file, 'rb') as f:
        tensor.ParseFromString(f.read())
    inputs.append(numpy_helper.to_array(tensor))

In [34]:
loaded_lib = tvm.runtime.load_module(lib_path)

TVMError: Traceback (most recent call last):
  3: TVMFuncCall
  2: std::_Function_handler<void (tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*), tvm::runtime::TypedPackedFunc<tvm::runtime::Module (std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > const&, std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > const&)>::AssignTypedLambda<tvm::runtime::Module (*)(std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > const&, std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > const&)>(tvm::runtime::Module (*)(std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > const&, std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > const&), std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >)::{lambda(tvm::runtime::TVMArgs const&, tvm::runtime::TVMRetValue*)#1}>::_M_invoke(std::_Any_data const&, tvm::runtime::

In [ ]:
#loaded_param = tvm.runtime.load_param_dict(open(param_path, "rb").read())
loaded_param = bytearray(open(param_path, "rb").read())

In [ ]:
# context

In [ ]:
ctx = tvm.cpu()

In [ ]:
module = tvm.contrib.graph_executor.GraphModule(loaded_lib["mnist"](ctx))

In [ ]:
#module = tvm.contrib.graph_executor.create(loaded_json, loaded_lib, ctx)

In [ ]:
module.load_params(loaded_param)

In [ ]:
module.set_input("Input3", inputs[0])

# tes

In [ ]:
module.run()

In [ ]:
out_deploy = module.get_output(0).asnumpy()

In [ ]:
print(out_deploy)